In [ ]:
from pyproj import Geod
from jinja2 import Template
from branca.element import MacroElement
from folium.plugins import MarkerCluster
import matplotlib.pyplot as MPlot
import pandas as Pd
import random
import geopandas as GPd
import folium as Folium
from shapely.geometry import Point

İncelediğim verini toplu taşıma problemlerini tam olarak belirlemek için yetersiz bir veri olduğundan toplu taşıma problemlerini değil ancak ulaşım problemi beklentilerini hesapladım.

İlk olarak toplu taşıma probleminden kastım ne : Herhangi bir toplu taşıma hattında yola çıkan otobüs, tramvay vb.. ulaşım araçlarının belirli saatlarde çok yoğun olması veya
trafik, yoğunluk vb.. sebeplerle o hattı kullanacak kişileri fazla bekletmesi ve bunlardan sebep mağduriyet yaratması durumları toplu taşımanın problemli olduğu durumlardır. Bunlara ek olarak bir hatta yola çıkan bir aracın
yeterli kitleye hizmet sunamaması ve aracın tabiri caizse boş gidip boş gelmeside bir toplu taşıma problemidir.

Yani üç durum toplu taşıma problemlerine sebebiyet verebilir :
    Ulaşım araçlarının fazla kalabalık (yoğun) olması.
    Ulaşım araçlarının hedef konuma (durak vb..) mağduriyet yaratacak kadar geç gitmesi.
    Ulaşım araçlarının yeterli veya hedef kitleye hizmet sunmaması, tarife mevcudiyetinin boşa olması.

Bu iki durumun yaşanıp yaşanmadığını belirlemede etkili olan unsurlar şöyle ki :
    Saatler : Bu durumların bir hat özelinde sürekli olarak yaşanması ; incelediğimiz verinin ait olduğu şehrin kültürel yapısı, fiziki yapısı, nufus yapısı vb.. etmenlerden çok mümkün görünmüyor. Şöyle ki veriyi incelediğimiz zamanda bunu
    net bir şekilde görebilmekteyiz. Örn : Gece toplu taşıma kullanımının nerdeyse olmaması, iş giriş-çıkış saatlerindeyse en yüksek seviyelerine ulaşması. Bu durum sebebiyle ve sağlanan verinin saatlik periyotlarla verilmesi
    sebebiyle veriyi saatlik periyotlarda inceliyeceğiz. Problem potansiyellerini de saatlere göre belirleyeceğiz.
    Konumlar : Toplu taşıma hatlarının her biri farklı konumlarda, bölgelerde halka hizmet sağlar. Hatların hizmet sağladığı bölgeler trafiği, nüfusu, sanayisi, tarımı, hizmet sektörü vb.. durumlarıyla birbirlerinden ayrışır ve her bir bölgeden
    ulaşım problemi beklentimizin farklı olmasına sebep olur. Bölgelerin bu durumları toplu taşıma hatlarındaki problemleri doğrudan etkilemelidir.
    Biniş Sayıları : Bir hatta kaç kişinin bindiği diğer faktörlerle olan etkileşimi göz önüne alınarak hesaba katılırsa en önemli değişkenlerden birisi olacaktır. Çünkü tüm faktörler en sonunda hattın kullanımını etkiler
    ve hattın kullanımı o hatta bir toplu taşıma problemi olup olmadığını belirleyen temel unsurdur.

# Toplu Ulaşım Elkart ve Banka Kartı Kullanım Verileri

In [ ]:
Elkart_Binis_2021 = Pd.read_csv('BinisSayilari/Elkart_Binis_2021.csv', sep = ';')
Elkart_Binis_2020_05_12 = Pd.read_csv('BinisSayilari/Elkart_Binis_2020_05_12.csv', sep = ';')
Elkart_Binis_2020_03_04 = Pd.read_csv('BinisSayilari/Elkart_Binis_2020_03_04.csv', sep = ';')
Elkart_Binis_2020_01_02 = Pd.read_csv('BinisSayilari/Elkart_Binis_2020_01_02.csv', sep = ';')
Elkart_Binis_2019 = Pd.read_csv('BinisSayilari/Elkart_Binis_2019.csv', sep = ';')

BankaKart_Binis_2021 = Pd.read_csv('BinisSayilari/BankaKart_Binis_2021.csv', sep = ';')
BankaKart_Binis_2020_05_12 = Pd.read_csv('BinisSayilari/BankaKart_Binis_2020_05_12.csv', sep = ';')
BankaKart_Binis_2020_03_04 = Pd.read_csv('BinisSayilari/BankaKart_Binis_2020_03_04.csv', sep = ';')
BankaKart_Binis_2020_01_02 = Pd.read_csv('BinisSayilari/BankaKart_Binis_2020_01_02.csv', sep = ';')
BankaKart_Binis_2019 = Pd.read_csv('BinisSayilari/BankaKart_Binis_2019.csv', sep = ';')

# Pandemi sonrası toplu taşımada basılan kart verileri
_data = Pd.concat([Elkart_Binis_2021, Elkart_Binis_2020_05_12, BankaKart_Binis_2021, BankaKart_Binis_2020_05_12]) 

# Pandemi öncesi toplu taşımada basılan kart verileri
#_data = Elkart_Binis_2019.merge(Elkart_Binis_2020_01_02).merge(BankaKart_Binis_2019).merge(BankaKart_Binis_2020_01_02) 

# Pandemi başı kapanma dönemi toplu taşımada basılan kart verileri (çok istisnai veriler içerdiğinden farklı bir küme kabul ettik)
#_data = Elkart_Binis_2020_03_04.merge(BankaKart_Binis_2020_03_04) 

TT_Binis = Pd.DataFrame(columns=['Hat', 'Gun', 'Saat', 'Binis'])
TT_Binis['Hat'] = _data['ANA_HAT_NO'].astype(str) + '_' + _data['ALT_HAT_NO'].astype(str) # Hatları "Ana Hat Numarası_Alt Hat Numarası" şeklinde unique isimlendiriyorum (İsimlendirme tüm projede böyle yapılacaktır.)
TT_Binis['Gun'] = Pd.to_datetime(_data['TARIH'].astype(str), format='%Y-%m-%d').dt.dayofweek # Tarih verisini haftanın gününe çeviriyorum : Pazartesi = 0, Salı = 1, Çarşamba = 2 ... gibi.
TT_Binis['Saat'] = _data['SAAT']
TT_Binis['Binis'] = _data['BINIS_SAYISI']

# Hat Gün Ve Saat değişkenlerine göre veriyi gruplayıp ortalamalarını alıyorum. Ortalama (mean) değeri bana hatlara hangi gün hangi saat ortalama kaç kişi bindiği değerini veriyor.
TT_Binis = TT_Binis.groupby(['Hat', 'Gun', 'Saat'])['Binis'].agg('mean').round(2).reset_index()

# Biniş sayılarını hafta içi, cumartesi ve pazar olarak ayırıyorum
TT_Binis_Haftaici = TT_Binis[TT_Binis['Gun'] < 5].groupby(['Hat', 'Saat']).agg('mean').reset_index().drop(['Gun'], axis=1)
TT_Binis_Cumartesi = TT_Binis[TT_Binis['Gun'] == 5].groupby(['Hat', 'Saat']).agg('mean').reset_index().drop(['Gun'], axis=1)
TT_Binis_Pazar = TT_Binis[TT_Binis['Gun'] == 6].groupby(['Hat', 'Saat']).agg('mean').reset_index().drop(['Gun'], axis=1)

# 10_0 tramvay hattıdır (Alaaddin - Kampüs İçi) ve proje boyunca veriler bu hattın hafta içi biniş verileri üzerinden izlenecektir.
Tramvay_Haftaici_Binis = TT_Binis_Haftaici[(TT_Binis_Haftaici['Hat'] == '10_0')]
MPlot.figure(figsize = (14,10))
MPlot.xticks(range(0,24,1))
MPlot.grid()
MPlot.plot(Tramvay_Haftaici_Binis.Saat, Tramvay_Haftaici_Binis.Binis)
Tramvay_Haftaici_Binis

# Toplu Ulaşım Lokasyon Verileri

In [ ]:
Mahalleler_Liste = GPd.GeoDataFrame(GPd.read_file('CografiVeriler/Mahalleler.geojson'))
Ilceler_Liste = GPd.GeoDataFrame(GPd.read_file('CografiVeriler/Ilceler.geojson'))

_Geod = Geod(ellps='WGS84')

for mahalleIndex, mahalle in Mahalleler_Liste.iterrows(): # Mahalle adları tek başına unique olmadığından hangi ilçelerde olduklarını buluyorum
    for ilceIndex, ilce in Ilceler_Liste.iterrows():
        if ilce['geometry'].contains( Point(mahalle['geometry'].centroid.xy[0][0], mahalle['geometry'].centroid.xy[1][0]) ):
            Mahalleler_Liste.loc[mahalleIndex , ['Mahalle', 'YuzOlcumu']] = [ilce['Ilce'] + ' - ' + mahalle['Mahalle'], int(abs(_Geod.geometry_area_perimeter(mahalle['geometry'])[0])) / 10**6 ]
            break

Mahalleler_Liste

In [ ]:
_data = Pd.read_csv('CografiVeriler/Durak_Konumlari.csv', delimiter=';', encoding='utf8')
_data['Durak_Kodu'] = _data['Durak_Kodu'].str.upper()

Duraklar = Pd.DataFrame(columns=['Durak_Isim', 'Durak_Kodu', 'Coords'])
Duraklar['Durak_Isim'] = _data['Durak_Isim']

Duraklar = GPd.GeoDataFrame( _data, geometry=GPd.points_from_xy(_data.LNG, _data.LAT)).drop(['LAT', 'LNG'], axis=1)

for indexDurak, Durak in Duraklar.iterrows(): # Durakları unique mahallelerle eşleştiriyorum
    for indexMahalle, Mahalle in Mahalleler_Liste.iterrows():
        if Mahalle['geometry'].contains(Durak['geometry']):
            Duraklar.loc[indexDurak, 'Mahalle'] = Mahalle['Mahalle']
            break

Duraklar[Duraklar['Durak_Isim'] == 'OTOGAR'].head()

In [ ]:
_data = Pd.read_csv('CografiVeriler/Mahalleler_Istatistik.csv', delimiter=';', encoding='windows-1254').fillna(0)

Mahalle_Istatistikleri = Pd.DataFrame(columns=['Mahalle', 'Nufus', 'Isyeri', 'Cadde'])
Mahalle_Istatistikleri['Mahalle'] = _data['Ilce'] + ' - ' + _data['Mahalle']
Mahalle_Istatistikleri['Nufus'] = _data['NUFUS'].astype(int)
Mahalle_Istatistikleri['Isyeri'] = (_data['ISYERI_SAYISI'] + _data['KAMU_ISYERI'] + _data['OZEL_ISYERI']).astype(int)
Mahalle_Istatistikleri['Cadde'] = _data['MAH_CD_SK_SAYISI']

Mahalle_Istatistikleri.head(25)

In [ ]:
Mahalleler = Pd.merge(Mahalleler_Liste, Mahalle_Istatistikleri, on=['Mahalle']).fillna(0)
Mahalleler['NufusYogunlugu'] = (Mahalleler['Nufus'] / Mahalleler['YuzOlcumu'])
Mahalleler['IsyeriYogunlugu'] = (Mahalleler['Isyeri'] / Mahalleler['YuzOlcumu'])
Mahalleler['CaddeYogunlugu'] = (Mahalleler['Cadde'] / Mahalleler['YuzOlcumu'])

Hastaneler = GPd.GeoDataFrame(GPd.read_file('CografiVeriler/Hastaneler.geojson'))
Okullar = GPd.GeoDataFrame(GPd.read_file('CografiVeriler/Okullar.geojson'))
Universite_Yerleskeleri = Pd.read_csv('CografiVeriler/Universite_Yerleskeleri.csv', delimiter=';', encoding='windows-1254')
Toplanma_Alanlari = Pd.read_csv('CografiVeriler/Toplanma_Alanlari.csv', delimiter=';', encoding='windows-1254')

Hastaneler['geometry'] = Hastaneler['geometry'].centroid
Okullar['geometry'] = Okullar['geometry'].centroid
Universite_Yerleskeleri = GPd.GeoDataFrame( Universite_Yerleskeleri, geometry=GPd.points_from_xy(Universite_Yerleskeleri.BOYLAM, Universite_Yerleskeleri.ENLEM)).drop(['ENLEM', 'BOYLAM'], axis=1)
Toplanma_Alanlari = GPd.GeoDataFrame( Toplanma_Alanlari ,geometry=GPd.points_from_xy(Toplanma_Alanlari.BOYLAM, Toplanma_Alanlari.ENLEM)).drop(['ENLEM', 'BOYLAM'], axis=1)
Konumlar = Pd.concat([Hastaneler, Okullar, Universite_Yerleskeleri, Toplanma_Alanlari])

Konumlar['DEGER'] = Konumlar['DEGER'].fillna(1)
Mahalleler['KonumDegeri'] = 0

for index, Konum in Konumlar.iterrows():
    for indexMahalle, Mahalle in Mahalleler.iterrows():
        if Mahalle['geometry'].contains(Konum['geometry']):
            Mahalleler.loc[indexMahalle, 'KonumDegeri'] += Konum['DEGER']
            break

Konumlar

In [ ]:
Mahalleler['Beklenti'] = ((Mahalleler['NufusYogunlugu'] - Mahalleler['NufusYogunlugu'].min()) / (Mahalleler['NufusYogunlugu'].max() - Mahalleler['NufusYogunlugu'].min()) * 50)                                                     + ((Mahalleler['IsyeriYogunlugu'] - Mahalleler['IsyeriYogunlugu'].min()) / (Mahalleler['IsyeriYogunlugu'].max() - Mahalleler['IsyeriYogunlugu'].min()) * 30)                                                                        + ((Mahalleler['KonumDegeri'] - Mahalleler['KonumDegeri'].min()) / (Mahalleler['KonumDegeri'].max() - Mahalleler['KonumDegeri'].min()) * 20)

Mahalleler.sort_values('Beklenti', ascending=False).head(25)

In [ ]:
_data = Pd.read_csv('CografiVeriler/Hat_Guzergahlari.csv', delimiter=';', encoding='windows-1254').fillna(0)

Guzergahlar = Pd.DataFrame(columns=['Hat', 'geometry'])
Guzergahlar['Hat'] = _data['ANA_HAT_NO'].astype(str) + '_' + _data['ALT_HAT_NO'].astype(str)
Guzergahlar['geometry'] = _data.reset_index()[['LAT', 'LNG']].values.tolist()
Guzergahlar = Guzergahlar.groupby(['Hat'])['geometry'].apply(list).reset_index()
Guzergahlar

# Toplu Taşıma Hatları Hakkındaki Veriler

In [ ]:
_data = Pd.read_csv('Hatlar/Tarifeler.csv', sep=';')

Hat_Tarifeleri = Pd.DataFrame(columns=['Hat', 'Saat', 'Dakika', 'Gun', 'Sayi'])
Hat_Tarifeleri['Hat'] = _data['HAT_NO'].astype(str) + '_' + _data['ALT_HAT_NO'].astype(str)
Hat_Tarifeleri['Saat'] = Pd.to_datetime(_data['SAAT'].astype(str), format='%H:%M').dt.hour
Hat_Tarifeleri['Dakika'] = Pd.to_datetime(_data['SAAT'].astype(str), format='%H:%M').dt.minute
Hat_Tarifeleri['Gun'] = _data['TARIFE_ZAMANI'] # Gün değeri şu şekilde => 'HaftaIci' : 'H', 'Cumartesi' : 'C', 'Pazar' : 'P'

# Veride tüm 
Hat_Tarifeleri = Hat_Tarifeleri.groupby(['Hat', 'Gun', 'Saat'])['Dakika'].apply(list).reset_index()

MPlot.figure(figsize = (14,10))
MPlot.xticks(range(0,24,1))
MPlot.grid()
MPlot.plot(Hat_Tarifeleri[(Hat_Tarifeleri['Hat'] == '10_1') & (Hat_Tarifeleri['Gun'] == 'H')]['Saat'], Hat_Tarifeleri[(Hat_Tarifeleri['Hat'] == '10_1') & (Hat_Tarifeleri['Gun'] == 'H')]['Dakika'].str.len())
Hat_Tarifeleri[(Hat_Tarifeleri['Hat'] == '10_1') & (Hat_Tarifeleri['Gun'] == 'H')]

In [ ]:
_data = Pd.read_csv('Hatlar/Duraklar.csv', sep = ';', encoding='windows-1254')

Hat_Duraklari = Pd.DataFrame(columns=['Hat', 'Durak'])
Hat_Duraklari['Hat'] = _data['ANA_HAT_NO'].astype(str) + '_' + _data['ALT_HAT_NO'].astype(str)
Hat_Duraklari['Durak'] = _data['DURAK_NO'].str.upper()
Hat_Duraklari = Hat_Duraklari.groupby(['Hat'])['Durak'].apply(list).reset_index()

Hat_Duraklari[Hat_Duraklari['Hat'] == '10_1']

In [ ]:
_data = Pd.read_csv('Hatlar/Durak_Varis_Sureleri.csv', sep = ';')

Durak_Varis_Sureleri = Pd.DataFrame(columns=['Hat', 'Durak', 'Saat', 'Sure', 'Gun'])
Durak_Varis_Sureleri['Hat'] = _data['ANA_HAT_NO'].astype(str) + '_' + _data['ALT_HAT_NO'].astype(str)
Durak_Varis_Sureleri['Gun'] = Pd.to_datetime(_data['VARIS_ZAMAN'].astype(str), format='%d.%m.%Y %H:%M').dt.dayofweek
Durak_Varis_Sureleri['Durak'] = _data['BITIS_DURAK_NO'].str.upper()
Durak_Varis_Sureleri['Saat'] = Pd.to_datetime(_data['VARIS_ZAMAN'].astype(str), format='%d.%m.%Y %H:%M').dt.hour
Durak_Varis_Sureleri['Sure'] = (Pd.to_datetime(_data['VARIS_ZAMAN'].astype(str), format='%d.%m.%Y %H:%M') - Pd.to_datetime(_data['CIKIS_ZAMAN'].astype(str), format='%d.%m.%Y %H:%M')).astype('timedelta64[m]').astype(int)
Durak_Varis_Sureleri.drop(Durak_Varis_Sureleri[Durak_Varis_Sureleri['Sure'] > 15].index, inplace = True)

# Hangi hat saat kaçta hangi durağa kaç dakikada gidiyor ortalamasını buluyorum. Elimdeki veri setinin küçüklüğü (1 haftalık veri) sebebiyle doğruluğu çok yüksek olmayacaktır.
# Örneğin tramvay hattı T10A durağına hafta içi saat 5te ortalama kaç dakikada gitmiş.
Durak_Varis_Sureleri_Haftaici = Pd.DataFrame(columns=['Hat', 'Saat', 'Durak', 'Sure'])
Durak_Varis_Sureleri_Haftaici = Durak_Varis_Sureleri[Durak_Varis_Sureleri['Gun'] < 5].drop('Gun', axis=1).groupby(['Hat', 'Durak', 'Saat'])['Sure'].agg('mean').round(2).reset_index()

Durak_Varis_Sureleri_Cumartesi = Pd.DataFrame(columns=['Hat', 'Saat', 'Durak', 'Sure'])
Durak_Varis_Sureleri_Cumartesi = Durak_Varis_Sureleri[Durak_Varis_Sureleri['Gun'] == 5].drop('Gun', axis=1).groupby(['Hat', 'Durak', 'Saat'])['Sure'].agg('mean').round(2).reset_index()

Durak_Varis_Sureleri_Pazar = Pd.DataFrame(columns=['Hat', 'Saat', 'Durak', 'Sure'])
Durak_Varis_Sureleri_Pazar = Durak_Varis_Sureleri[Durak_Varis_Sureleri['Gun'] == 6].drop('Gun', axis=1).groupby(['Hat', 'Durak', 'Saat'])['Sure'].agg('mean').round(2).reset_index()

print ( 
' Sure >= 2 : %' + str(round((len(Durak_Varis_Sureleri_Haftaici[Durak_Varis_Sureleri_Haftaici['Sure'] >= 2])/len(Durak_Varis_Sureleri_Haftaici) * 100), 2)) + '\n' +
' 2 > Sure > 1 : %' + str(round((len(Durak_Varis_Sureleri_Haftaici[(Durak_Varis_Sureleri_Haftaici['Sure'] > 1) & (Durak_Varis_Sureleri_Haftaici['Sure'] < 2)])/len(Durak_Varis_Sureleri_Haftaici) * 100), 2)) + '\n' +
' 1 >= Sure >= 0.5 : %' + str(round((len(Durak_Varis_Sureleri_Haftaici[(Durak_Varis_Sureleri_Haftaici['Sure'] >= 0.5) & (Durak_Varis_Sureleri_Haftaici['Sure'] <= 1)])/len(Durak_Varis_Sureleri_Haftaici) * 100), 2)) + '\n' +
' Sure < 0.5 : %' + str(round((len(Durak_Varis_Sureleri_Haftaici[Durak_Varis_Sureleri_Haftaici['Sure'] < 0.5])/len(Durak_Varis_Sureleri_Haftaici) * 100), 2))
)

MPlot.figure(figsize = (14,10))
MPlot.xticks(range(0,24,1))
MPlot.grid()
MPlot.plot(Durak_Varis_Sureleri_Haftaici[Durak_Varis_Sureleri_Haftaici['Hat'] == '10_2'].head(19).Saat, Durak_Varis_Sureleri_Haftaici[Durak_Varis_Sureleri_Haftaici['Hat'] == '10_2'].head(19).Sure)
Durak_Varis_Sureleri_Haftaici[Durak_Varis_Sureleri_Haftaici['Hat'] == '10_2'].head(19)


In [ ]:
# Hangi hat hangi durağa ortalama kaç dakikada gitmiş. (Saat değişkenini veriden çıkartıp ortalama değer buluyorum.)

Durak_Varis_Sureleri_Haftaici_Ortalama = Durak_Varis_Sureleri_Haftaici.drop(['Saat'], axis=1).groupby(['Hat', 'Durak'])['Sure'].agg('mean').round(2).reset_index()
Durak_Varis_Sureleri_Cumartesi_Ortalama = Durak_Varis_Sureleri_Cumartesi.drop(['Saat'], axis=1).groupby(['Hat', 'Durak'])['Sure'].agg('mean').round(2).reset_index()
Durak_Varis_Sureleri_Pazar_Ortalama = Durak_Varis_Sureleri_Pazar.drop(['Saat'], axis=1).groupby(['Hat', 'Durak'])['Sure'].agg('mean').round(2).reset_index()

Durak_Varis_Sureleri_Haftaici_Ortalama[Durak_Varis_Sureleri_Haftaici_Ortalama['Hat'] == '10_0'].head(24)

In [ ]:
# Hangi hat hangi ortalama kaç dakikada durak değiştirmiş. (Durak değişkenini veriden çıkartıyorum ama saat verisi duruyor. Veriyi görselleştirmeye hazırlıyorum.)

Hat_Varis_Sureleri_Haftaici_Ortalama = Durak_Varis_Sureleri_Haftaici.drop(['Durak'], axis=1).groupby(['Hat', 'Saat'])['Sure'].agg('mean').round(2).reset_index()
Hat_Varis_Sureleri_Cumartesi_Ortalama = Durak_Varis_Sureleri_Cumartesi.drop(['Durak'], axis=1).groupby(['Hat', 'Saat'])['Sure'].agg('mean').round(2).reset_index()
Hat_Varis_Sureleri_Pazar_Ortalama = Durak_Varis_Sureleri_Pazar.drop(['Durak'], axis=1).groupby(['Hat', 'Saat'])['Sure'].agg('mean').round(2).reset_index()

MPlot.figure(figsize = (14,10))
MPlot.xticks(range(0,24,1))
MPlot.grid()
MPlot.plot(Hat_Varis_Sureleri_Haftaici_Ortalama[Hat_Varis_Sureleri_Haftaici_Ortalama['Hat'] == '10_2'].head(24).Saat, Hat_Varis_Sureleri_Haftaici_Ortalama[Hat_Varis_Sureleri_Haftaici_Ortalama['Hat'] == '10_2'].head(24).Sure)
Hat_Varis_Sureleri_Haftaici_Ortalama[Hat_Varis_Sureleri_Haftaici_Ortalama['Hat'] == '10_2'].head(24)

In [ ]:
# Hatta o anda yolda bulunan araçlar o saatte kaç durağa uğradı verisini çıkartıyorum çünkü elkart biniş sayıları verisi 1 saatlik periyotlarla hat bazlı verilmiş durumda
Hat_Gidilen_Durak_Sayilari_Ortalama_Haftaici = Pd.DataFrame(columns=['Hat', 'Saat', 'Durak'])
Hat_Gidilen_Durak_Sayilari_Ortalama_Haftaici = Durak_Varis_Sureleri[Durak_Varis_Sureleri['Gun'] < 5].drop(['Sure', 'Gun'], axis=1)
Hat_Gidilen_Durak_Sayilari_Ortalama_Haftaici = Hat_Gidilen_Durak_Sayilari_Ortalama_Haftaici.groupby(['Hat', 'Saat']).count().divide(5).round(2).reset_index()

Hat_Gidilen_Durak_Sayilari_Ortalama_Cumartesi = Pd.DataFrame(columns=['Hat', 'Saat', 'Durak'])
Hat_Gidilen_Durak_Sayilari_Ortalama_Cumartesi = Durak_Varis_Sureleri[Durak_Varis_Sureleri['Gun'] < 5].drop(['Sure', 'Gun'], axis=1)
Hat_Gidilen_Durak_Sayilari_Ortalama_Cumartesi = Hat_Gidilen_Durak_Sayilari_Ortalama_Cumartesi.groupby(['Hat', 'Saat']).count().round(2).reset_index()

Hat_Gidilen_Durak_Sayilari_Ortalama_Pazar = Pd.DataFrame(columns=['Hat', 'Saat', 'Durak'])
Hat_Gidilen_Durak_Sayilari_Ortalama_Pazar = Durak_Varis_Sureleri[Durak_Varis_Sureleri['Gun'] < 5].drop(['Sure', 'Gun'], axis=1)
Hat_Gidilen_Durak_Sayilari_Ortalama_Pazar = Hat_Gidilen_Durak_Sayilari_Ortalama_Pazar.groupby(['Hat', 'Saat']).count().round(2).reset_index()

MPlot.figure(figsize = (14,10))
MPlot.xticks(range(0,24,1))
MPlot.grid()
MPlot.plot(Hat_Gidilen_Durak_Sayilari_Ortalama_Haftaici[Hat_Gidilen_Durak_Sayilari_Ortalama_Haftaici['Hat'] == '10_0'].Saat, Hat_Gidilen_Durak_Sayilari_Ortalama_Haftaici[Hat_Gidilen_Durak_Sayilari_Ortalama_Haftaici['Hat'] == '10_0'].Durak)
Hat_Gidilen_Durak_Sayilari_Ortalama_Haftaici[Hat_Gidilen_Durak_Sayilari_Ortalama_Haftaici['Hat'] == '10_0']

Mahallelere göre beklenti değişkenini kullanarak durakların o hattaki değer katsayılarıyla binen kişi ortalamasının korelasyonuna bakılacak

# Görselleştirme

In [ ]:
class BindColormap(MacroElement):
    """Binds a colormap to a given layer.

    Parameters
    ----------
    colormap : branca.colormap.ColorMap
        The colormap to bind.
    """
    def __init__(self, layer, colormap):
        super(BindColormap, self).__init__()
        self.layer = layer
        self.colormap = colormap
        self._template = Template(u"""
        {% macro script(this, kwargs) %}
            {{this.colormap.get_name()}}.svg[0][0].style.display = 'block';
            {{this._parent.get_name()}}.on('overlayadd', function (eventLayer) {
                if (eventLayer.layer == {{this.layer.get_name()}}) {
                    {{this.colormap.get_name()}}.svg[0][0].style.display = 'block';
                }});
            {{this._parent.get_name()}}.on('overlayremove', function (eventLayer) {
                if (eventLayer.layer == {{this.layer.get_name()}}) {
                    {{this.colormap.get_name()}}.svg[0][0].style.display = 'none';
                }});
        {% endmacro %}
        """)

def ColorMap(col, capt):
        return Folium.branca.colormap.LinearColormap(colors=[Folium.branca.colormap._parse_hex('#440154'),
                                                            Folium.branca.colormap._parse_hex('#414487'),
                                                            Folium.branca.colormap._parse_hex('#2A788E'),
                                                            Folium.branca.colormap._parse_hex('#22A884'),
                                                            Folium.branca.colormap._parse_hex('#7AD151'),
                                                            Folium.branca.colormap._parse_hex('#FDE725')],
                                                caption=capt,
                                                vmin=col.min(), 
                                                vmax=col.max())
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']

def Add_Layer_Choropleth(Map, dataframe, column_name, gruop_title, colormap_title, tooltip_fields, tooltip_aliases):
    LayerGroup = Folium.FeatureGroup(gruop_title, show=False)
    LayerColorMap = ColorMap(dataframe[column_name], colormap_title)
    Layer = Folium.features.GeoJson(
        data = dataframe,
        name=gruop_title,
        style_function= lambda x: {"weight" : 0.5, 'color' : 'black', 'fillColor' : LayerColorMap(x['properties'][column_name]), 'fillOpacity' : 0.75},
        highlight_function= lambda x: {'fillColor': '#000000', 'color':'#000000', 'fillOpacity': 0.75, 'weight': 0.1}, 
        tooltip=Folium.features.GeoJsonTooltip(
            fields=tooltip_fields,
            aliases=tooltip_aliases,
            style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
        )
    )
    LayerGroup.add_child(Layer)
    Map.add_child(LayerGroup).add_child(LayerColorMap).add_child(BindColormap(LayerGroup, LayerColorMap))

def Add_Layer_Points(Map, dataframe, gruop_title, popup_value):
    LayerGroup = Folium.FeatureGroup(gruop_title, show=False)
    LayerMarkerCluster = MarkerCluster(name=gruop_title)
    for indexPoint, Point in dataframe.iterrows():
        Folium.features.Marker(
            [Point['geometry'].coords[0][1], Point['geometry'].coords[0][0]],
            popup=Point[popup_value]
        ).add_to(LayerMarkerCluster)
    LayerGroup.add_child(LayerMarkerCluster)
    Map.add_child(LayerGroup)

def Add_Layer_Lines(Map, dataframe, gruop_title, tooltip_fields):
    LayerGroup = Folium.FeatureGroup(gruop_title, show=False)
    for indexLine, Line in dataframe.iterrows():
        Folium.PolyLine(
            Line['geometry'],
            color=random.choice(colors),
            name=gruop_title,
            highlight_function= lambda x: {'fillColor': '#000000', 'color':'#000000', 'fillOpacity': 0.75, 'weight': 0.1},
            tooltip=Line[tooltip_fields]
        ).add_to(LayerGroup)
    Map.add_child(LayerGroup)

In [ ]:
Map = Folium.Map(location=[37.872482, 32.492237], zoom_start=15, zoom_max=20)

Add_Layer_Choropleth(Map, Mahalleler, 'NufusYogunlugu', 'Nüfus Yoğunluğu', 'Nüfus Yoğunluğu', ['Mahalle', 'Nufus', 'YuzOlcumu'], ['Mahalle', 'Nüfus', 'Yuz Olcumu (Km^2)'])
Add_Layer_Choropleth(Map, Mahalleler, 'IsyeriYogunlugu', 'İşyeri Yoğunluğu', 'İşyeri Yoğunluğu', ['Mahalle', 'Isyeri', 'YuzOlcumu'], ['Mahalle', 'İşyeri Sayısı', 'Yuz Olcumu (Km^2)'])
Add_Layer_Choropleth(Map, Mahalleler, 'Beklenti', 'Yoğunluk Beklentisi', 'Yoğunluk Beklentisi', ['Mahalle', 'Beklenti', 'YuzOlcumu'], ['Mahalle', 'Beklenti (0-100)', 'Yuz Olcumu (Km^2)'])
Add_Layer_Points(Map, Duraklar, 'Duraklar', 'Durak_Isim')
Add_Layer_Points(Map, Konumlar, 'Konumlar', 'ADI')
Add_Layer_Lines(Map, Guzergahlar, 'Güzergahlar', 'Hat')

Map.add_child(Folium.LayerControl())
Map